# KEA

Author: Moshe C. Silverstein  
Date: 07-2017

In [1]:
import sys
import numpy as np
import pandas as pd
import Utilities
from Utilities import *
from importlib import reload

# Load Data

Data Retrieved 2/1/2017 

http://amp.pharm.mssm.edu/X2K/datasets/KEA.zip (KEA)

In [2]:
kea_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Input/KEA/phosphorylation_reactions.csv', sep=',', header=None)

# Load Gmene Mapping File

In [3]:
mappingFile = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Doc and Mapping/mappingFile_2017.txt', sep='\t', header=None, index_col=0)

# Select Relevent Data

In [4]:
kea_df = kea_df[[2,3,4,5]]

In [5]:
kea_df.rename(columns={2: 'Protein A (gene name)', 3:'Protein B (gene name)', 4:'PubMed ID', 5:'Source databases'}, inplace=True)

In [6]:
# change dataset so each pubmed id is associated with one interaction
lstPub = []
lstPA = []
lstPB = []

for i,index in enumerate(kea_df.index):
    
    progressPercent = ((i+1)/len(kea_df.index))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(kea_df.index)))
    sys.stdout.flush()
    
    for pub in kea_df.ix[index, 'PubMed ID'].split(';'):
        lstPub.append(pub)
        lstPA.append(kea_df.ix[index, 'Protein A (gene name)'])
        lstPB.append(kea_df.ix[index, 'Protein B (gene name)'])

kea_df_better = pd.DataFrame()
kea_df_better['Protein A (gene name)'] = lstPA
kea_df_better['Protein B (gene name)'] = lstPB
kea_df_better['PubMed ID'] = lstPub

In [7]:
kea_df = kea_df_better.copy()

# Map Gene to human and updated approved symbols

In [8]:
mapgenesymbols(kea_df, mappingFile, 'Protein A (gene name)', 'Protein B (gene name)')

# Unfiltered

In [9]:
# uf = unfiltered
kea_uf_df = kea_df.copy()

### Combine duplicate ppis while concatanatig referances

In [10]:
combineDupPPIs(kea_uf_df)

### Create .sig File

In [11]:
pathU = '~/./Documents/Harmonizome/PPI Library/Output/IndividualResources/Unfiltered/'

createSigFile(kea_uf_df, pathU, 'kea', False)

# Filtered

In [12]:
kea_f_df = kea_df.copy()

In [13]:
# drop any data that was published with more then 10 PPI's per publication or doesnt have a ppi
filterPPIbyPubmed(kea_f_df, 10)

### Combine duplicate ppis while concatanatig referances

In [14]:
combineDupPPIs(kea_f_df)

### Create .sig File

In [15]:
pathF = '~/./Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/'

createSigFile(kea_f_df, pathF, 'kea', True)